## Variational Quantum Boltzmann Machine - Qiskit Implementation 

### Imports

In [17]:
# General Imports
import numpy as np
from scipy.linalg import expm

# Circuit Imports
from qiskit.circuit.library import RealAmplitudes, EfficientSU2
from qiskit.circuit import Parameter

# Operator Imports
from qiskit.aqua.operators import I, Z, StateFn, CircuitStateFn, SummedOp
# from qiskit.aqua.operators.gradients import NaturalGradient

# Additional Imports
from qiskit.quantum_info import state_fidelity, partial_trace, Statevector
from qiskit.aqua.components.optimizers import SPSA, CG, ADAM, COBYLA

## Gibbs state preparation using VarQITE

### Define the system parameters and initialize an Ansatz

$\rho^{Gibbs} = \frac{e^H/{k_BT}}{Z}$

Given:
    
    class BM -> H
    H -> f(x)
    
    f(x) -> L(x)
    d/dx L(x)
    

In [18]:
# Temperature
k_BT = 1

# Evolution time
t =  1/(2*k_BT)

# Define the model Hamiltonian 
H = SummedOp([0.3 * Z^Z^ I^I, 0.2 * Z^I^ I^I,  0.5 * I^Z^ I^I]) 

### Define the system parameters and initialize an Ansatz

$\rho^{Gibbs} = \frac{e^H/{k_BT}}{Z}$

In [19]:
# Instantiate the model ansatz
depth = 1
entangler_map = [[i+1, i] for i in range(H.num_qubits - 1)]
ansatz = EfficientSU2(4, reps=depth, entanglement = entangler_map)
qr = ansatz.qregs[0]
for i in range(int(len(qr)/2)):
    ansatz.cx(qr[i], qr[i+int(len(qr)/2)])
    
# Initialize the Ansatz parameters
param_values_init = np.zeros(2* H.num_qubits * (depth + 1))
for j in range(2 * H.num_qubits * depth, int(len(param_values_init) - H.num_qubits - 2)):
    param_values_init[int(j)] = np.pi/2.
    

#### Initial State

The Ansatz $|\psi\left(\omega\left(\tau\right)\right)\rangle$ is initialized such that the first two qubits are in a maximally mixed state.

In [20]:
print('Initial parameters ', param_values_init)

# Initial State

print('\n Circuit ', ansatz.assign_parameters(dict(zip(ansatz.ordered_parameters, param_values_init))))

print('\n Full statevector ', CircuitStateFn(ansatz.assign_parameters \
                                          (dict(zip(ansatz.ordered_parameters, param_values_init)))).eval().primitive.data)

print('\n Maximally mixed state', partial_trace(CircuitStateFn(ansatz.assign_parameters\
                        (dict(zip(ansatz.ordered_parameters, param_values_init)))).eval().primitive.data, [0, 1]).data)

Initial parameters  [0.         0.         0.         0.         0.         0.
 0.         0.         1.57079633 1.57079633 0.         0.
 0.         0.         0.         0.        ]

 Circuit       ┌─────────┐┌─────────┐┌───┐┌──────────┐┌─────────┐                       »
q_0: ┤ RY(0.0) ├┤ RZ(0.0) ├┤ X ├┤ RY(pi/2) ├┤ RZ(0.0) ├───────────────────────»
     ├─────────┤├─────────┤└─┬─┘└──┬───┬───┘├─────────┴┐┌─────────┐           »
q_1: ┤ RY(0.0) ├┤ RZ(0.0) ├──■─────┤ X ├────┤ RY(pi/2) ├┤ RZ(0.0) ├───────────»
     ├─────────┤├─────────┤        └─┬─┘    └──┬───┬───┘├─────────┤┌─────────┐»
q_2: ┤ RY(0.0) ├┤ RZ(0.0) ├──────────■─────────┤ X ├────┤ RY(0.0) ├┤ RZ(0.0) ├»
     ├─────────┤├─────────┤                    └─┬─┘    ├─────────┤├─────────┤»
q_3: ┤ RY(0.0) ├┤ RZ(0.0) ├──────────────────────■──────┤ RY(0.0) ├┤ RZ(0.0) ├»
     └─────────┘└─────────┘                             └─────────┘└─────────┘»
«               
«q_0: ──■───────
«       │       
«q_1: ──┼────■──
«     ┌─┴─┐  │  


#### Let's define the target observable consisting of the Ansatz and the Hamiltonian

$$ \langle \psi\left(\omega\left(\tau\right)\right)|H|\psi\left(\omega\left(\tau\right)\right)\rangle $$

In [21]:
# Define the Hamiltonian as observable w.r.t. the wavefunction generated by the Ansatz   
# Use statevector simulation
ansatz_op = StateFn(ansatz)
op = ~StateFn(H) @ ansatz_op

print('\n Energy expectation value of the initial state ', op.assign_parameters(dict(zip(ansatz.ordered_parameters, param_values_init))).eval())


 Energy expectation value of the initial state  0j


#### Target state

$\rho^{target} = \frac{e^{H\otimes I}/{k_BT}}{Z}$

In [22]:
# Compute the density matrix corresponding to the target Gibbs state
h_mat = H.to_matrix()
gibbs_target = expm(-h_mat*t) / np.trace(expm(-h_mat*t))
gibbs_target = partial_trace(gibbs_target, [0, 1]).data

print('Target state ', gibbs_target)

Target state  [[0.14517971+0.j 0.        +0.j 0.        +0.j 0.        +0.j]
 [0.        +0.j 0.32310338+0.j 0.        +0.j 0.        +0.j]
 [0.        +0.j 0.        +0.j 0.23936087+0.j 0.        +0.j]
 [0.        +0.j 0.        +0.j 0.        +0.j 0.29235603+0.j]]


#### Gradient

Given a parameterized quantum state `|ψ(θ)〉 = V(θ)|ψ〉` with input state `|ψ〉` and parametrized Ansatz `V(θ)`, and an Operator `O(ω)` we want to compute 
``` math
d⟨ψ(θ)|O(ω)|ψ(θ)〉/ dθ
```

In [24]:
param_dict = dict(zip(ansatz.ordered_parameters, np.random.rand(len(ansatz.ordered_parameters))))

'''
Note that this is a simple implementation that only works for Pauli rotations 
and does not account for product or chain rules.
'''

def gradient(op, parameter_dict):
    gradient = []
    for param in list(parameter_dict.keys()):
        parameter_dict[param] += np.pi/2
        p_shift = op.assign_parameters(param_dict).eval()
        parameter_dict[param] -= np.pi
        m_shift = op.assign_parameters(param_dict).eval()
        parameter_dict[param] += np.pi /2
        gradient += [0.5 * (p_shift - m_shift)]
    return gradient

print('Gradient', gradient(op, param_dict))

Gradient [(-0.5816035852342102+3.4000000000000004e-17j), (-0.17837575268045774+4.2000000000000005e-17j), (0.00549210488667462+1.0500000000000001e-17j), (-0.1211652912009978+1.55e-17j), (0.02668452653744935-6.35e-17j), (-0.02091054530861644-5.8e-17j), (-0.000516891666519359-6.85e-17j), (-0.0036154248219966156-5.6e-17j), (-0.5195263206440762-3.25e-17j), (-0.07516060003225247+7.499999999999996e-18j), (-0.011803895307625524-6.500000000000001e-18j), (-0.14724427233477894-3.5000000000000014e-18j), (2.7755575615628914e-17-4.6500000000000004e-17j), (2.7755575615628914e-17-5.5e-17j), (5.551115123125783e-17+5.25e-17j), (5.551115123125783e-17+5.3500000000000007e-17j)]


#### Quantum Fisher Information (QFI)

The `QFI` is a metric tensor which is representative for the representation capacity of a parameterized quantum state `|ψ(θ)〉 = V(θ)|ψ〉` generated by an input state `|ψ〉` and a parametrized Ansatz `V(θ)`.

The entries of the `QFI` for a pure state reads
```
[QFI]kl= Re[〈∂kψ|∂lψ〉−〈∂kψ|ψ〉〈ψ|∂lψ〉] * 0.25.
``` 

In [25]:
'''
Implementation according to https://arxiv.org/pdf/2008.06517.pdf
Note that this is a simple implementation that only works for Pauli rotations 
and does not account for product or chain rules.
'''

def QFI(state_op, parameter_dict):
    meas = ~state_op.assign_parameters(parameter_dict)
    qfi = np.zeros((len(parameter_dict), len(parameter_dict)))
    for i, param_i in enumerate(list(parameter_dict.keys())):
        for j, param_j in enumerate(list(parameter_dict.keys())):
            parameter_dict[param_i] += np.pi /2
            parameter_dict[param_j] += np.pi /2
            qfi_op = meas @ state_op.assign_parameters(parameter_dict)
            qfi[i, j] += qfi_op.eval() / 4
            parameter_dict[param_j] -= np.pi
            qfi_op = meas @ state_op.assign_parameters(parameter_dict)
            qfi[i, j] -= qfi_op.eval() / 4
            parameter_dict[param_i] -= np.pi
            qfi_op = meas @ state_op.assign_parameters(parameter_dict)
            qfi[i, j] += qfi_op.eval() / 4
            parameter_dict[param_j] += np.pi
            qfi_op = meas @ state_op.assign_parameters(parameter_dict)
            qfi[i, j] -= qfi_op.eval() / 4
            parameter_dict[param_i] += np.pi/2
            parameter_dict[param_j] -= np.pi/2
    return(qfi)
            
            
            
print('QFI', QFI(ansatz_op, param_dict))

<ipython-input-25-a8583345413d>:13: ComplexWarning: Casting complex values to real discards the imaginary part
  qfi[i, j] += qfi_op.eval() / 4
<ipython-input-25-a8583345413d>:16: ComplexWarning: Casting complex values to real discards the imaginary part
  qfi[i, j] -= qfi_op.eval() / 4
<ipython-input-25-a8583345413d>:19: ComplexWarning: Casting complex values to real discards the imaginary part
  qfi[i, j] += qfi_op.eval() / 4
<ipython-input-25-a8583345413d>:22: ComplexWarning: Casting complex values to real discards the imaginary part
  qfi[i, j] -= qfi_op.eval() / 4


QFI [[-5.00000000e-01 -1.38777878e-17  1.38777878e-17 -2.77555756e-17
   0.00000000e+00  2.77555756e-17  2.77555756e-17  0.00000000e+00
  -4.23459324e-01  2.13567673e-02  2.46542142e-04  1.24086652e-04
  -3.85644128e-02 -9.16402862e-03 -5.02166255e-05 -5.93337598e-05]
 [-1.38777878e-17 -5.00000000e-01  0.00000000e+00  0.00000000e+00
   2.77555756e-17 -5.55111512e-17  5.55111512e-17  5.55111512e-17
  -1.38777878e-17 -1.07365635e-01  1.31959013e-03  6.64160375e-04
  -8.32667268e-17 -4.90494713e-02 -2.68779053e-04 -3.17577527e-04]
 [ 0.00000000e+00  0.00000000e+00 -5.00000000e-01  1.38777878e-17
   2.77555756e-17  0.00000000e+00  2.77555756e-17  2.77555756e-17
   0.00000000e+00  0.00000000e+00 -4.33202308e-02  8.82624373e-03
   0.00000000e+00  0.00000000e+00 -3.57189245e-03 -4.22039128e-03]
 [-2.77555756e-17  0.00000000e+00  1.38777878e-17 -5.00000000e-01
   2.77555756e-17  2.77555756e-17  0.00000000e+00  2.77555756e-17
   0.00000000e+00  0.00000000e+00  0.00000000e+00 -7.92996947e-04
  -

#### Quantum Natural Gradient (QNG)

The `QNG` combines the state resp. probability gradients and the metric tensor which is representative for the representation capabilities of a parameterized quantum state `|ψ(θ)〉 = V(θ)|ψ〉` generated by an input state `|ψ〉` and a parametrized Ansatz `V(θ)`.

The `QNG` reads
```
QNG = QFI^-1 d⟨ψ(θ)|O(ω)|ψ(θ)〉/ dθ
```
Now, if `O(ω) = H` then QNG is equivalent to the propagation rule applied in VarQITE.

In [41]:
def QNG(op, state_op, parameter_dict):
    grad = gradient(op, parameter_dict)
    qfi = QFI(state_op, parameter_dict)
    alpha = 1e-7
    # Use regularization
    qng, _, _, _ = np.linalg.lstsq(qfi * alpha * np.diag(qfi), grad, rcond=None)
    return qng
    

### Define the parameter propagation rule according to McLachlan's variational principle

In [42]:
'''
The following implementation is based on the following PR https://github.com/Qiskit/qiskit-aqua/pull/1293
'''

def get_gibbs_state_params(op, ansatz, param_values, time_steps):

    # Convert the operator that holds the Hamiltonian and ansatz into a NaturalGradient operator 
    # nat_grad = NaturalGradient().convert(op, ansatz.ordered_parameters, method = 'lin_comb', regularization = 'ridge')
    # Propagate the Ansatz parameters step by step according to the explicit Euler method
    for step in time_steps:
        param_dict = dict(zip(ansatz.ordered_parameters, param_values))
        # nat_grad_result = np.real(nat_grad.assign_parameters(param_dict).eval())
        nat_grad_result = QNG(op, ansatz_op, param_dict)
        param_values = list(np.subtract(param_values, t/num_time_steps * np.real(nat_grad_result)))
    return param_values


### Run the parameter propagation

In [43]:
# Define the discretization grid of the time steps
num_time_steps = 10
time_steps = np.linspace(0, t, num_time_steps)
param_values = get_gibbs_state_params(op, ansatz, param_values_init, time_steps)
    
print('Final parameter values', param_values)

<ipython-input-25-a8583345413d>:13: ComplexWarning: Casting complex values to real discards the imaginary part
  qfi[i, j] += qfi_op.eval() / 4
<ipython-input-25-a8583345413d>:16: ComplexWarning: Casting complex values to real discards the imaginary part
  qfi[i, j] -= qfi_op.eval() / 4
<ipython-input-25-a8583345413d>:19: ComplexWarning: Casting complex values to real discards the imaginary part
  qfi[i, j] += qfi_op.eval() / 4
<ipython-input-25-a8583345413d>:22: ComplexWarning: Casting complex values to real discards the imaginary part
  qfi[i, j] -= qfi_op.eval() / 4


Final parameter values [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.5707963267948966, 1.5707963267948966, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


#### Check the fidelity between trained and target state

In [44]:
# Compute the density matrix corresponding to the final Gibbs state    
gibbs_state = Statevector.from_instruction(ansatz.assign_parameters(dict(zip(ansatz.ordered_parameters, param_values))))
gibbs_state = partial_trace(gibbs_state, [0, 1])
print('Gibbs state ', np.around(gibbs_state.data, 3))

print('Target state ', np.around(gibbs_target.data, 3))

# Evaluate the fidelity between the trained and the target state
fidelity = state_fidelity(np.around(gibbs_target.data, 3), np.around(gibbs_state.data, 3), validate=False)

print('Fidelity between trained and target state ', fidelity)

Gibbs state  [[0.25+0.j 0.  +0.j 0.  +0.j 0.  +0.j]
 [0.  +0.j 0.25+0.j 0.  +0.j 0.  +0.j]
 [0.  +0.j 0.  +0.j 0.25+0.j 0.  +0.j]
 [0.  +0.j 0.  +0.j 0.  +0.j 0.25+0.j]]
Target state  [[0.145+0.j 0.   +0.j 0.   +0.j 0.   +0.j]
 [0.   +0.j 0.323+0.j 0.   +0.j 0.   +0.j]
 [0.   +0.j 0.   +0.j 0.239+0.j 0.   +0.j]
 [0.   +0.j 0.   +0.j 0.   +0.j 0.292+0.j]]
Fidelity between trained and target state  0.9784830576761623


## Train a generative QBM 
More explicitly, we train here a fully visible, diagonal, generative QBM using gradient-free optimization.

### Initialize a parameterized Hamiltonian and the target PDF

In [45]:
a = Parameter('a')    
b = Parameter('b')
c = Parameter('c')  

# Define the model Hamiltonian with parameters
H = SummedOp([a * Z^Z^I^I, b * Z^I^ I ^ I, c * I ^ Z^ I ^ I]) 

# Define the target PDF
p_target = [0.5, 0, 0, 0.5] 

#### Define the loss function and the optimizer

In [46]:
# Define the loss function
def loss(H_coeffs):
    H_op = H.assign_parameters(dict(zip([a, b, c], np.real(H_coeffs))))
    
    #Combine the measurement and ansatz operator
    op = ~StateFn(H_op) @ ansatz_op
    
    #Prepare the Gibbs state
    param_values = get_gibbs_state_params(op, ansatz, param_values_init, time_steps)
    p_qbm = ansatz_op.assign_parameters(dict(zip(ansatz.ordered_parameters, param_values))).eval().primitive
    p_qbm = np.diag(partial_trace(p_qbm, [0, 1]).data)
    print('Trained probability ', p_qbm)
    loss_fn = -np.sum(np.multiply(p_target, np.log(p_qbm)))
#     print(loss_fn)
    return np.real(loss_fn)

### Customize
Try different optimizers

In [47]:
optimizer = SPSA(maxiter = 50)

### Train the QBM

In [ ]:
result = optimizer.optimize(3, loss, initial_point = ([-2 , .2, .5]))
print('Trained parameters ', result[0])

<ipython-input-25-a8583345413d>:13: ComplexWarning: Casting complex values to real discards the imaginary part
  qfi[i, j] += qfi_op.eval() / 4
<ipython-input-25-a8583345413d>:16: ComplexWarning: Casting complex values to real discards the imaginary part
  qfi[i, j] -= qfi_op.eval() / 4
<ipython-input-25-a8583345413d>:19: ComplexWarning: Casting complex values to real discards the imaginary part
  qfi[i, j] += qfi_op.eval() / 4
<ipython-input-25-a8583345413d>:22: ComplexWarning: Casting complex values to real discards the imaginary part
  qfi[i, j] -= qfi_op.eval() / 4


Trained probability  [0.25+0.j 0.25+0.j 0.25+0.j 0.25+0.j]
Trained probability  [0.25+0.j 0.25+0.j 0.25+0.j 0.25+0.j]


In [ ]:
#Construct the Hamiltonian with the final parameterw
H_op = H.assign_parameters(dict(zip([a, b, c], [-1.29810568, -0.04292338,  0.1362023])))

#Combine the measurement and ansatz operator
op = ~StateFn(H_op) @ ansatz_op

#Prepare the Gibbs state
param_values = get_gibbs_state_params(op, ansatz, param_values_init, time_steps)

# Get the sampling probabilities
p_qbm = ansatz_op.assign_parameters(dict(zip(ansatz.ordered_parameters, param_values))).eval().primitive
p_qbm = np.diag(partial_trace(p_qbm, [0, 1]).data)

# Evaluate the l1 norm between the trained and the target state
norm = np.linalg.norm(p_target-p_qbm, ord = 1)

print('L1-norm between trained and target distributions ', norm)

In [ ]:
print(p_qbm)